<a href="https://colab.research.google.com/github/JamillyNichele/TOBIAS-BOT-PRO-/blob/LInk-Colab/Tobias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =============================================================================
# 📦 CÉLULA 1: INSTALAÇÃO DE DEPENDÊNCIAS
# Execute PRIMEIRO e aguarde conclusão
# =============================================================================

import subprocess
import sys
from IPython.display import HTML, display, clear_output

print("🚀 Iniciando instalação do Tobias AI...")

packages = [
    'transformers',
    'torch',
    'accelerate',
    'sentencepiece',
    'openai-whisper',
    'yt-dlp',
    'ipywidgets'
]

for i, pkg in enumerate(packages, 1):
    print(f"[{i}/{len(packages)}] Instalando {pkg}...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])

print("\n📦 Instalando FFmpeg...")
subprocess.run(["apt", "update", "-qq"], capture_output=True)
subprocess.run(["apt", "install", "-y", "ffmpeg"], capture_output=True)

clear_output()
display(HTML("""
<div style="background: linear-gradient(135deg, #10b981, #059669);
            color: white; padding: 25px; border-radius: 15px; text-align: center;">
    <h2>✅ Instalação Concluída!</h2>
    <p style="margin: 10px 0;">Todas as dependências foram instaladas</p>
    <p style="font-size: 14px; opacity: 0.9;">👉 Execute a CÉLULA 2</p>
</div>
"""))

In [ ]:
# =============================================================================
# 📚 CÉLULA 2: IMPORTS
# Execute após CÉLULA 1
# =============================================================================

import os
import time
import warnings
import glob
from datetime import datetime
from threading import Thread
from typing import Dict, Optional
from collections import Counter

import yt_dlp
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

warnings.filterwarnings("ignore")

display(HTML("""
<div style="background: #dbeafe; color: #1e40af; padding: 15px;
            border-radius: 8px; text-align: center;">
    ✅ Imports carregados | 👉 Execute a CÉLULA 3
</div>
"""))

In [ ]:
# =============================================================================
# 🎨 CÉLULA 5: INTERFACE GRÁFICA
# Execute após CÉLULA 4
# =============================================================================

class TobiasInterface:
    """Interface do Tobias AI"""

    def __init__(self):
        self.core = TobiasCore()
        self.processor = VideoProcessor()
        self.processing = False
        self._build_ui()

    def _build_ui(self):
        """Constrói interface"""

        # CSS
        self.css = HTML("""
        <style>
            .tobias-header {
                background: linear-gradient(135deg, #667eea, #764ba2);
                color: white; padding: 30px; border-radius: 15px;
                text-align: center; margin: 20px 0;
            }
            .section-box {
                background: #f8fafc; padding: 20px; border-radius: 10px;
                margin: 15px 0; border-left: 4px solid #667eea;
            }
            .status-success {
                background: #d1fae5; color: #065f46; padding: 12px;
                border-radius: 8px; margin: 10px 0;
            }
            .status-error {
                background: #fee2e2; color: #991b1b; padding: 12px;
                margin: 10px 0; border-radius: 8px;
            }
            .info-box {
                background: #dbeafe; color: #1e40af; padding: 15px;
                border-radius: 8px; margin: 10px 0;
            }
        </style>
        """)

        # Header
        self.header = HTML("""
        <div class="tobias-header">
            <h1>🤖 Tobias AI</h1>
            <p>Análise Inteligente de Vídeos com IA Local</p>
            <div style="margin-top: 10px;">
                <span style="background: rgba(255,255,255,0.2); padding: 5px 12px;
                             border-radius: 12px; font-size: 13px; margin: 0 5px;">
                    ✨ Sem API Keys
                </span>
                <span style="background: rgba(255,255,255,0.2); padding: 5px 12px;
                             border-radius: 12px; font-size: 13px; margin: 0 5px;">
                    🚀 100% Gratuito
                </span>
                <span style="background: rgba(255,255,255,0.2); padding: 5px 12px;
                             border-radius: 12px; font-size: 13px; margin: 0 5px;">
                    🔒 Privado
                </span>
            </div>
        </div>
        """)

        # Info
        self.info = HTML("""
        <div class="info-box">
            <strong>ℹ️ Como funciona:</strong><br>
            • Primeiro use: O modelo de IA será baixado automaticamente (~250MB)<br>
            • Roda localmente no seu Colab (sem internet após download)<br>
            • Análises ilimitadas e gratuitas<br>
            • Seus dados nunca saem do ambiente
        </div>
        """)

        # Botão carregar modelo
        self.load_model_btn = widgets.Button(
            description='⚡ Carregar Modelo de IA',
            button_style='primary',
            icon='download',
            layout=widgets.Layout(width='300px', height='45px')
        )
        self.load_model_btn.on_click(self._load_model)

        # Modo de análise
        self.mode_dropdown = widgets.Dropdown(
            options=[
                ('📝 Resumo Rápido', 'summary'),
                ('🔍 Análise Detalhada', 'detailed'),
                ('🎓 Modo Educacional', 'educational'),
                ('💼 Análise de Negócios', 'business')
            ],
            value='summary',
            description='Modo:',
            style={'description_width': '100px'},
            layout=widgets.Layout(width='100%')
        )

        # URL
        self.url_input = widgets.Text(
            placeholder='Cole a URL do YouTube aqui...',
            description='URL:',
            style={'description_width': '100px'},
            layout=widgets.Layout(width='70%')
        )

        self.process_btn = widgets.Button(
            description='🚀 Processar Vídeo',
            button_style='success',
            icon='play',
            layout=widgets.Layout(width='28%', height='40px')
        )
        self.process_btn.on_click(self._process)

        # Quiz
        self.quiz_btn = widgets.Button(
            description='❓ Gerar Quiz do Vídeo',
            button_style='warning',
            icon='question',
            layout=widgets.Layout(width='48%', height='40px')
        )
        self.quiz_btn.on_click(self._generate_quiz)

        # Chat sobre o vídeo
        self.chat_btn = widgets.Button(
            description='💬 Conversar sobre o Vídeo',
            button_style='info',
            icon='comments',
            layout=widgets.Layout(width='48%', height='40px')
        )
        self.chat_btn.on_click(self._toggle_chat)

        # Área de chat (inicialmente oculta)
        self.chat_input = widgets.Text(
            placeholder='Digite sua pergunta sobre o vídeo...',
            description='Pergunte:',
            style={'description_width': '70px'},
            layout=widgets.Layout(width='75%', display='none')
        )

        self.chat_send_btn = widgets.Button(
            description='Enviar',
            button_style='primary',
            icon='paper-plane',
            layout=widgets.Layout(width='22%', display='none')
        )
        self.chat_send_btn.on_click(self._send_chat)

        self.chat_output = widgets.Output(layout=widgets.Layout(display='none'))
        self.chat_active = False

        # Progress com label
        self.progress_label = widgets.HTML(
            value='<div style="text-align: center; color: #64748b; font-size: 14px; margin: 5px 0;">Pronto para processar</div>'
        )

        self.progress = widgets.FloatProgress(
            value=0, min=0, max=100, bar_style='info',
            layout=widgets.Layout(width='100%', height='30px')
        )

        self.status = widgets.HTML()
        self.output = widgets.Output()
        self.last_transcription = None

        # Add separators and spaces
        self.sep1 = widgets.HTML("<hr>")
        self.sep2 = widgets.HTML("<hr>")
        self.sep3 = widgets.HTML("<hr>")
        self.sep4 = widgets.HTML("<hr>")
        self.space1 = widgets.VBox([], layout=widgets.Layout(height='10px'))
        self.space2 = widgets.VBox([], layout=widgets.Layout(height='10px'))

    def _load_model(self, btn):
        """Carrega modelo"""
        btn.disabled = True
        btn.description = "Carregando..."
        self._update_status("📥 Baixando modelo de IA... Isso pode levar alguns minutos na primeira vez.", "info")

        if self.core.load_model():
            self._update_status("✅ Modelo carregado! Tobias está pronto para análises.", "success")
            btn.description = "✅ Modelo Carregado"
            btn.button_style = 'success'
        else:
            self._update_status("❌ Erro ao carregar modelo. Tente novamente.", "error")
            btn.description = "⚡ Tentar Novamente"
            btn.disabled = False

    def _update_status(self, msg: str, type: str = "info"):
        """Atualiza status"""
        css_class = "status-success" if type == "success" else ("status-error" if type == "error" else "info-box")
        self.status.value = f'<div class="{css_class}">{msg}</div>'

    def _update_progress(self, value: int, label: str = ""):
        """Atualiza progresso com label"""
        self.progress.value = value

        # Cor da barra baseada no progresso
        if value < 30:
            color = "#3b82f6"  # Azul
        elif value < 70:
            color = "#f59e0b"  # Laranja
        else:
            color = "#10b981"  # Verde

        self.progress_label.value = f'''
        <div style="text-align: center; margin: 5px 0;">
            <div style="color: {color}; font-weight: 600; font-size: 16px;">
                {value}%
            </div>
            <div style="color: #64748b; font-size: 13px; margin-top: 3px;">
                {label if label else "Processando..."}
            </div>
        </div>
        '''

    def _process(self, btn):
        """Processa vídeo - VERSÃO SIMPLIFICADA"""
        if self.processing:
            return

        url = self.url_input.value.strip()
        if not url:
            self._update_status("❌ Cole a URL do YouTube", "error")
            return

        if 'youtube.com' not in url and 'youtu.be' not in url:
            self._update_status("❌ URL inválida. Use uma URL do YouTube.", "error")
            return

        # Confirmar que quer processar
        self.processing = True
        btn.disabled = True
        btn.description = "Processando..."

        self._update_progress(0, "Iniciando...")
        self._update_status("🚀 Iniciando processamento...", "info")

        # Executar DIRETO (sem thread para debug)
        self._do_process_direct(url)

    def _do_process_direct(self, url: str):
        """Processamento DIRETO sem threads"""
        try:
            print("\n" + "="*50)
            print("INICIANDO PROCESSAMENTO")
            print("="*50)

            # Fase 1: Download
            self._update_progress(5, "Baixando do YouTube...")
            self._update_status("📥 Conectando ao YouTube...", "info")

            print("\n[FASE 1] Download do YouTube")
            audio_file, metadata = self.processor.download_audio(url)

            self._update_progress(35, "Download completo!")
            self._update_status(f"✅ Áudio baixado: {metadata['duration']//60}min", "success")
            time.sleep(1)

            # Fase 2: Transcrição
            self._update_progress(40, "Transcrevendo com Whisper...")
            self._update_status("🎤 Transcrevendo áudio...", "info")

            print("\n[FASE 2] Transcrição com Whisper")
            transcription = self.processor.transcribe(audio_file)
            self.last_transcription = transcription

            self._update_progress(70, "Transcrição completa!")
            self._update_status(f"✅ {len(transcription.split())} palavras", "success")
            time.sleep(1)

            # Fase 3: Análise
            self._update_progress(75, "Analisando com IA...")
            self._update_status("🤖 Analisando conteúdo...", "info")

            print("\n[FASE 3] Análise com IA")
            analysis = self.core.analyze(transcription, self.mode_dropdown.value)

            self._update_progress(90, "Análise completa!")
            time.sleep(0.5)

            # Fase 4: Resultados
            self._update_progress(95, "Gerando resultados...")
            self._update_status("📊 Preparando visualização...", "info")

            print("\n[FASE 4] Gerando resultados")
            self._show_results(metadata, transcription, analysis)

            # Cleanup
            print("\n[LIMPEZA] Removendo arquivos temporários")
            self.processor.cleanup(audio_file)

            self._update_progress(100, "✅ Concluído!")
            self._update_status("✅ Processamento concluído com sucesso! 🎉", "success")

            print("\n" + "="*50)
            print("PROCESSAMENTO CONCLUÍDO")
            print("="*50 + "\n")

        except Exception as e:
            print(f"\n❌ ERRO: {str(e)}\n")
            self._update_progress(0, "Erro")
            self._update_status(f"❌ Erro: {str(e)}", "error")

            # Dicas específicas
            if "download" in str(e).lower():
                self._update_status("💡 Tente: Vídeo mais curto, URL pública, ou reinicie o Colab", "error")

        finally:
            self.processing = False
            self.process_btn.disabled = False
            self.process_btn.description = "🚀 Processar Vídeo"

    def _generate_quiz(self, btn):
        """Gera quiz"""
        if not self.last_transcription:
            self._update_status("❌ Processe um vídeo primeiro para gerar o quiz.", "error")
            return

        btn.disabled = True
        self._update_status("🎲 Gerando quiz personalizado...", "info")

        quiz = self.core.generate_quiz(self.last_transcription)

        with self.output:
            if quiz['success']:
                display(HTML(f"""
                <div class="section-box" style="background: #fef3c7; border-left-color: #f59e0b;">
                    <h3>❓ Quiz Gerado</h3>
                    <pre style="white-space: pre-wrap; line-height: 1.8;
                                font-family: inherit; background: white;
                                padding: 15px; border-radius: 8px;">{quiz['quiz']}</pre>
                </div>
                """))

        btn.disabled = False
        self._update_status("✅ Quiz gerado com sucesso!", "success")

    def _toggle_chat(self, btn):
        """Ativa/desativa modo chat"""
        if not self.last_transcription:
            self._update_status("❌ Processe um vídeo primeiro para conversar sobre ele.", "error")
            return

        self.chat_active = not self.chat_active

        if self.chat_active:
            # Ativar chat
            btn.description = '💬 Fechar Chat'
            btn.button_style = 'danger'
            self.chat_input.layout.display = 'flex'
            self.chat_send_btn.layout.display = 'flex'
            self.chat_output.layout.display = 'block'

            with self.chat_output:
                clear_output()
                display(HTML("""
                <div style="background: #f0f9ff; padding: 15px; border-radius: 8px;
                            border-left: 4px solid #3b82f6; margin: 10px 0;">
                    <strong>💬 Modo Conversação Ativado!</strong><br>
                    <p style="margin: 5px 0; color: #1e40af;">
                        Faça perguntas sobre o vídeo. Exemplos:<br>
                        • "Qual é o tema principal?"<br>
                        • "Resume os pontos mais importantes"<br>
                        • "O que aprendi com isso?"
                    </p>
                </div>
                """))
        else:
            # Desativar chat
            btn.description = '💬 Conversar sobre o Vídeo'
            btn.button_style = 'info'
            self.chat_input.layout.display = 'none'
            self.chat_send_btn.layout.display = 'none'
            self.chat_output.layout.display = 'none'

    def _send_chat(self, btn):
        """Envia mensagem no chat"""
        message = self.chat_input.value.strip()
        if not message:
            return

        btn.disabled = True
        self.chat_input.disabled = True

        with self.chat_output:
            display(HTML(f"""
            <div style="background: #e0f2fe; padding: 10px; border-radius: 8px;
                        margin: 5px 0; text-align: right;">
                <strong>Você:</strong> {message}
            </div>
            """))

            display(HTML("""
            <div style="background: #f1f5f9; padding: 10px; border-radius: 8px;
                        margin: 5px 0;">
                <strong>Tobias:</strong> <em>Pensando...</em>
            </div>
            """))

        # Gera resposta
        response = self.core.chat(message)

        with self.chat_output:
            # Remove "pensando..."
            clear_output(wait=True)

            # Mostra conversa completa
            for msg in self.core.conversation_history:
                display(HTML(f"""
                <div style="background: #e0f2fe; padding: 10px; border-radius: 8px;
                            margin: 5px 0; text-align: right;">
                    <strong>Você:</strong> {msg['user']}
                </div>
                <div style="background: #f1f5f9; padding: 10px; border-radius: 8px;
                            margin: 5px 0;">
                    <strong>Tobias:</strong> {msg['bot']}
                </div>
                """))

        self.chat_input.value = ''
        self.chat_input.disabled = False
        btn.disabled = False


    def _show_results(self, metadata: dict, transcription: str, analysis: dict):
        """Mostra resultados"""

        # Define contexto para conversação
        self.core.set_video_context(transcription, metadata)


        with self.output:
            clear_output()

            html = f"""
            <div class="section-box">
                <h2 style="color: #1e293b;">📹 {metadata['title']}</h2>
                <p style="color: #64748b; margin: 10px 0;">
                    <strong>Canal:</strong> {metadata['channel']} |
                    <strong>Views:</strong> {metadata['views']:,} |
                    <strong>Duração:</strong> {metadata['duration']//60}min {metadata['duration']%60}s
                </p>

                <hr style="border: none; border-top: 2px solid #e5e7eb; margin: 20px 0;">

                <h3 style="color: #475569;">📝 Transcrição ({len(transcription.split())} palavras)</h3>
                <details style="margin: 10px 0;">
                    <summary style="cursor: pointer; color: #667eea; font-weight: 600;
                                    padding: 10px; background: #f1f5f9; border-radius: 6px;">
                        📄 Clique para ver transcrição completa
                    </summary>
                    <div style="background: white; padding: 15px; border-radius: 8px;
                                margin-top: 10px; line-height: 1.8; max-height: 400px;
                                overflow-y: auto; border: 1px solid #e5e7eb;">
                        {transcription}
                    </div>
                </details>
            """

            if analysis['success']:
                html += f"""
                <hr style="border: none; border-top: 2px solid #e5e7eb; margin: 20px 0;">

                <div style="background: linear-gradient(135deg, #e0f2fe 0%, #bae6fd 100%);
                            padding: 20px; border-radius: 12px; margin: 15px 0;">
                    <div style="display: flex; justify-content: space-between; align-items: center; margin-bottom: 15px;">
                        <h3 style="color: #0369a1; margin: 0;">🤖 Análise Completa</h3>
                        <span style="background: #0369a1; color: white; padding: 6px 14px;
                                     border-radius: 15px; font-size: 12px; font-weight: 600;">
                            {analysis.get('provider', 'IA')}
                        </span>
                    </div>
                    <div style="background: white; padding: 20px; border-radius: 8px;
                                line-height: 1.8; white-space: pre-wrap; font-size: 15px;">
{analysis['analysis']}
                    </div>
                </div>
                """

            html += "</div>"
            display(HTML(html))

    def display(self):
        """Exibe interface"""
        # Encapsulate HTML objects in Output widgets
        css_output = widgets.Output()
        with css_output:
            display(self.css)

        header_output = widgets.Output()
        with header_output:
            display(self.header)

        info_output = widgets.Output()
        with info_output:
            display(self.info)


        layout = widgets.VBox([
            css_output,
            header_output,
            info_output,

            self.sep1,
            self.load_model_btn,

            self.sep2,
            self.mode_dropdown,

            self.sep3,
            widgets.HBox([self.url_input, self.process_btn]),

            self.sep4,
            widgets.HBox([self.quiz_btn, self.chat_btn]),

            # Área de chat
            self.space1,
            widgets.HBox([self.chat_input, self.chat_send_btn]),
            self.chat_output,

            self.space2,
            self.progress_label,
            self.progress,
            self.status,
            self.output
        ])

        display(layout)

display(HTML("""
<div style="background: #dbeafe; color: #1e40af; padding: 15px;
            border-radius: 8px; text-align: center;">
    ✅ TobiasInterface carregada | 👉 Execute a CÉLULA 6
</div>
"""))

In [ ]:
# =============================================================================
# 🎥 CÉLULA 4: PROCESSADOR DE VÍDEO
# Execute após CÉLULA 3
# =============================================================================

class VideoProcessor:
    """Processa vídeos do YouTube - Versão Simplificada e RÁPIDA"""

    def __init__(self):
        self._whisper = None
        self._device = self._detect_device()

    def _detect_device(self):
        """Detecta se há GPU disponível"""
        try:
            import torch
            return "cuda" if torch.cuda.is_available() else "cpu"
        except:
            return "cpu"

    def download_audio(self, url: str) -> tuple:
        """Download ULTRA SIMPLIFICADO - só o que funciona"""

        # Limpar tudo primeiro
        print("Limpando arquivos antigos...")
        import glob
        for pattern in ['audio.*', '*.mp3', '*.webm', '*.m4a']:
            for f in glob.glob(pattern):
                try:
                    os.remove(f)
                except:
                    pass

        # Configuração MÍNIMA e funcional
        ydl_opts = {
            'format': 'worstaudio',
            'outtmpl': 'audio',
            'quiet': False,
            'no_warnings': False,
        }

        print(f"Iniciando download de: {url}")

        try:
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                print("Extraindo informações...")
                info = ydl.extract_info(url, download=False)

                # Limitar duração
                duration = info.get('duration', 0)
                if duration > 1200:  # 20 minutos
                    raise Exception(f"Vídeo muito longo ({duration//60}min). Use vídeos de até 20 minutos.")

                metadata = {
                    'title': info.get('title', 'Sem título')[:80],
                    'channel': info.get('channel', 'Desconhecido'),
                    'duration': duration,
                    'views': info.get('view_count', 0)
                }

                print(f"Baixando: {metadata['title']} ({duration//60}min)")
                ydl.download([url])
                print("Download completo!")

        except Exception as e:
            raise Exception(f"Erro no download do YouTube: {str(e)}")

        # Procurar arquivo baixado
        print("Procurando arquivo de áudio...")
        audio_files = glob.glob('audio*')

        if not audio_files:
            raise Exception("Nenhum arquivo de áudio foi baixado. Verifique a URL.")

        audio_file = audio_files[0]
        print(f"✅ Arquivo encontrado: {audio_file}")

        return audio_file, metadata

    def transcribe(self, audio_path: str) -> str:
        """Transcrição SIMPLIFICADA"""
        try:
            if self._whisper is None:
                import whisper
                print("Carregando Whisper tiny...")
                self._whisper = whisper.load_model("tiny")
                print(f"✅ Whisper carregado!")

            print(f"Transcrevendo {audio_path}...")

            # SIMPLES e FUNCIONAL
            result = self._whisper.transcribe(
                audio_path,
                language='pt',
                verbose=False
            )

            text = result["text"]
            print(f"✅ Transcrição: {len(text.split())} palavras")
            return text

        except Exception as e:
            raise Exception(f"Erro na transcrição: {str(e)}")

    def cleanup(self, audio_path: str):
        """Remove arquivos"""
        try:
            import glob
            for f in glob.glob('audio*'):
                try:
                    os.remove(f)
                    print(f"Removido: {f}")
                except:
                    pass
        except:
            pass

display(HTML("""
<div style="background: #dbeafe; color: #1e40af; padding: 15px;
            border-radius: 8px; text-align: center;">
    ✅ VideoProcessor carregado | 👉 Execute a CÉLULA 5
</div>
"""))

In [ ]:
# =============================================================================
# 🎨 CÉLULA 5: INTERFACE GRÁFICA
# Execute após CÉLULA 4
# =============================================================================

class TobiasInterface:
    """Interface do Tobias AI"""

    def __init__(self):
        self.core = TobiasCore()
        self.processor = VideoProcessor()
        self.processing = False
        self._build_ui()

    def _build_ui(self):
        """Constrói interface"""

        # CSS
        self.css = HTML("""
        <style>
            .tobias-header {
                background: linear-gradient(135deg, #667eea, #764ba2);
                color: white; padding: 30px; border-radius: 15px;
                text-align: center; margin: 20px 0;
            }
            .section-box {
                background: #f8fafc; padding: 20px; border-radius: 10px;
                margin: 15px 0; border-left: 4px solid #667eea;
            }
            .status-success {
                background: #d1fae5; color: #065f46; padding: 12px;
                border-radius: 8px; margin: 10px 0;
            }
            .status-error {
                background: #fee2e2; color: #991b1b; padding: 12px;
                margin: 10px 0; border-radius: 8px;
            }
            .info-box {
                background: #dbeafe; color: #1e40af; padding: 15px;
                border-radius: 8px; margin: 10px 0;
            }
        </style>
        """)

        # Header
        self.header = HTML("""
        <div class="tobias-header">
            <h1>🤖 Tobias AI</h1>
            <p>Análise Inteligente de Vídeos com IA Local</p>
            <div style="margin-top: 10px;">
                <span style="background: rgba(255,255,255,0.2); padding: 5px 12px;
                             border-radius: 12px; font-size: 13px; margin: 0 5px;">
                    ✨ Sem API Keys
                </span>
                <span style="background: rgba(255,255,255,0.2); padding: 5px 12px;
                             border-radius: 12px; font-size: 13px; margin: 0 5px;">
                    🚀 100% Gratuito
                </span>
                <span style="background: rgba(255,255,255,0.2); padding: 5px 12px;
                             border-radius: 12px; font-size: 13px; margin: 0 5px;">
                    🔒 Privado
                </span>
            </div>
        </div>
        """)

        # Info
        self.info = HTML("""
        <div class="info-box">
            <strong>ℹ️ Como funciona:</strong><br>
            • Primeiro use: O modelo de IA será baixado automaticamente (~250MB)<br>
            • Roda localmente no seu Colab (sem internet após download)<br>
            • Análises ilimitadas e gratuitas<br>
            • Seus dados nunca saem do ambiente
        </div>
        """)

        # Botão carregar modelo
        self.load_model_btn = widgets.Button(
            description='⚡ Carregar Modelo de IA',
            button_style='primary',
            icon='download',
            layout=widgets.Layout(width='300px', height='45px')
        )
        self.load_model_btn.on_click(self._load_model)

        # Modo de análise
        self.mode_dropdown = widgets.Dropdown(
            options=[
                ('📝 Resumo Rápido', 'summary'),
                ('🔍 Análise Detalhada', 'detailed'),
                ('🎓 Modo Educacional', 'educational'),
                ('💼 Análise de Negócios', 'business')
            ],
            value='summary',
            description='Modo:',
            style={'description_width': '100px'},
            layout=widgets.Layout(width='100%')
        )

        # URL
        self.url_input = widgets.Text(
            placeholder='Cole a URL do YouTube aqui...',
            description='URL:',
            style={'description_width': '100px'},
            layout=widgets.Layout(width='70%')
        )

        self.process_btn = widgets.Button(
            description='🚀 Processar Vídeo',
            button_style='success',
            icon='play',
            layout=widgets.Layout(width='28%', height='40px')
        )
        self.process_btn.on_click(self._process)

        # Quiz
        self.quiz_btn = widgets.Button(
            description='❓ Gerar Quiz',
            button_style='warning',
            icon='question',
            layout=widgets.Layout(width='48%', height='40px')
        )
        self.quiz_btn.on_click(self._generate_quiz)

        # Chat
        self.chat_btn = widgets.Button(
            description='💬 Conversar',
            button_style='info',
            icon='comments',
            layout=widgets.Layout(width='48%', height='40px')
        )
        self.chat_btn.on_click(self._toggle_chat)

        # Área de chat
        self.chat_input = widgets.Text(
            placeholder='Digite sua pergunta...',
            description='Pergunte:',
            style={'description_width': '70px'},
            layout=widgets.Layout(width='75%', display='none')
        )

        self.chat_send_btn = widgets.Button(
            description='Enviar',
            button_style='primary',
            icon='paper-plane',
            layout=widgets.Layout(width='22%', display='none')
        )
        self.chat_send_btn.on_click(self._send_chat)

        self.chat_output = widgets.Output(layout=widgets.Layout(display='none'))
        self.chat_active = False

        # Progress
        self.progress_label = widgets.HTML(
            value='<div style="text-align: center; color: #64748b; font-size: 14px; margin: 5px 0;">Pronto para processar</div>'
        )

        self.progress = widgets.FloatProgress(
            value=0, min=0, max=100, bar_style='info',
            layout=widgets.Layout(width='100%', height='30px')
        )

        self.status = widgets.HTML()
        self.output = widgets.Output()
        self.last_transcription = None

        # Modo de análise
        self.mode_dropdown = widgets.Dropdown(
            options=[
                ('📝 Resumo Rápido', 'summary'),
                ('🔍 Análise Detalhada', 'detailed'),
                ('🎓 Modo Educacional', 'educational'),
                ('💼 Análise de Negócios', 'business')
            ],
            value='summary',
            description='Modo:',
            style={'description_width': '100px'},
            layout=widgets.Layout(width='100%')
        )

        # URL
        self.url_input = widgets.Text(
            placeholder='Cole a URL do YouTube aqui...',
            description='URL:',
            style={'description_width': '100px'},
            layout=widgets.Layout(width='70%')
        )

        self.process_btn = widgets.Button(
            description='🚀 Processar Vídeo',
            button_style='success',
            icon='play',
            layout=widgets.Layout(width='28%', height='40px')
        )
        self.process_btn.on_click(self._process)

        # Quiz
        self.quiz_btn = widgets.Button(
            description='❓ Gerar Quiz do Vídeo',
            button_style='warning',
            icon='question',
            layout=widgets.Layout(width='48%', height='40px')
        )
        self.quiz_btn.on_click(self._generate_quiz)

        # Chat sobre o vídeo
        self.chat_btn = widgets.Button(
            description='💬 Conversar sobre o Vídeo',
            button_style='info',
            icon='comments',
            layout=widgets.Layout(width='48%', height='40px')
        )
        self.chat_btn.on_click(self._toggle_chat)

        # Área de chat (inicialmente oculta)
        self.chat_input = widgets.Text(
            placeholder='Digite sua pergunta sobre o vídeo...',
            description='Pergunte:',
            style={'description_width': '70px'},
            layout=widgets.Layout(width='75%', display='none')
        )

        self.chat_send_btn = widgets.Button(
            description='Enviar',
            button_style='primary',
            icon='paper-plane',
            layout=widgets.Layout(width='22%', display='none')
        )
        self.chat_send_btn.on_click(self._send_chat)

        self.chat_output = widgets.Output(layout=widgets.Layout(display='none'))
        self.chat_active = False

        # Progress com label
        self.progress_label = widgets.HTML(
            value='<div style="text-align: center; color: #64748b; font-size: 14px; margin: 5px 0;">Pronto para processar</div>'
        )

        self.progress = widgets.FloatProgress(
            value=0, min=0, max=100, bar_style='info',
            layout=widgets.Layout(width='100%', height='30px')
        )

        self.status = widgets.HTML()
        self.output = widgets.Output()
        self.last_transcription = None

    def _load_model(self, btn):
        """Carrega modelo"""
        btn.disabled = True
        btn.description = "Carregando..."
        self._update_status("📥 Baixando modelo de IA... Isso pode levar alguns minutos na primeira vez.", "info")

        if self.core.load_model():
            self._update_status("✅ Modelo carregado! Tobias está pronto para análises.", "success")
            btn.description = "✅ Modelo Carregado"
            btn.button_style = 'success'
        else:
            self._update_status("❌ Erro ao carregar modelo. Tente novamente.", "error")
            btn.description = "⚡ Tentar Novamente"
            btn.disabled = False

    def _update_status(self, msg: str, type: str = "info"):
        """Atualiza status"""
        css_class = "status-success" if type == "success" else ("status-error" if type == "error" else "info-box")
        self.status.value = f'<div class="{css_class}">{msg}</div>'

    def _update_progress(self, value: int, label: str = ""):
        """Atualiza progresso com label"""
        self.progress.value = value

        # Cor da barra baseada no progresso
        if value < 30:
            color = "#3b82f6"  # Azul
        elif value < 70:
            color = "#f59e0b"  # Laranja
        else:
            color = "#10b981"  # Verde

        self.progress_label.value = f'''
        <div style="text-align: center; margin: 5px 0;">
            <div style="color: {color}; font-weight: 600; font-size: 16px;">
                {value}%
            </div>
            <div style="color: #64748b; font-size: 13px; margin-top: 3px;">
                {label if label else "Processando..."}
            </div>
        </div>
        '''

    def _process(self, btn):
        """Processa vídeo - VERSÃO SIMPLIFICADA"""
        if self.processing:
            return

        url = self.url_input.value.strip()
        if not url:
            self._update_status("❌ Cole a URL do YouTube", "error")
            return

        if 'youtube.com' not in url and 'youtu.be' not in url:
            self._update_status("❌ URL inválida. Use uma URL do YouTube.", "error")
            return

        # Confirmar que quer processar
        self.processing = True
        btn.disabled = True
        btn.description = "Processando..."

        self._update_progress(0, "Iniciando...")
        self._update_status("🚀 Iniciando processamento...", "info")

        # Executar DIRETO (sem thread para debug)
        self._do_process_direct(url)

    def _do_process_direct(self, url: str):
        """Processamento DIRETO sem threads"""
        try:
            print("\n" + "="*50)
            print("INICIANDO PROCESSAMENTO")
            print("="*50)

            # Fase 1: Download
            self._update_progress(5, "Baixando do YouTube...")
            self._update_status("📥 Conectando ao YouTube...", "info")

            print("\n[FASE 1] Download do YouTube")
            audio_file, metadata = self.processor.download_audio(url)

            self._update_progress(35, "Download completo!")
            self._update_status(f"✅ Áudio baixado: {metadata['duration']//60}min", "success")
            time.sleep(1)

            # Fase 2: Transcrição
            self._update_progress(40, "Transcrevendo com Whisper...")
            self._update_status("🎤 Transcrevendo áudio...", "info")

            print("\n[FASE 2] Transcrição com Whisper")
            transcription = self.processor.transcribe(audio_file)
            self.last_transcription = transcription

            self._update_progress(70, "Transcrição completa!")
            self._update_status(f"✅ {len(transcription.split())} palavras", "success")
            time.sleep(1)

            # Fase 3: Análise
            self._update_progress(75, "Analisando com IA...")
            self._update_status("🤖 Analisando conteúdo...", "info")

            print("\n[FASE 3] Análise com IA")
            analysis = self.core.analyze(transcription, self.mode_dropdown.value)

            self._update_progress(90, "Análise completa!")
            time.sleep(0.5)

            # Fase 4: Resultados
            self._update_progress(95, "Gerando resultados...")
            self._update_status("📊 Preparando visualização...", "info")

            print("\n[FASE 4] Gerando resultados")
            self._show_results(metadata, transcription, analysis)

            # Cleanup
            print("\n[LIMPEZA] Removendo arquivos temporários")
            self.processor.cleanup(audio_file)

            self._update_progress(100, "✅ Concluído!")
            self._update_status("✅ Processamento concluído com sucesso! 🎉", "success")

            print("\n" + "="*50)
            print("PROCESSAMENTO CONCLUÍDO")
            print("="*50 + "\n")

        except Exception as e:
            print(f"\n❌ ERRO: {str(e)}\n")
            self._update_progress(0, "Erro")
            self._update_status(f"❌ Erro: {str(e)}", "error")

            # Dicas específicas
            if "download" in str(e).lower():
                self._update_status("💡 Tente: Vídeo mais curto, URL pública, ou reinicie o Colab", "error")

        finally:
            self.processing = False
            self.process_btn.disabled = False
            self.process_btn.description = "🚀 Processar Vídeo"

    def _generate_quiz(self, btn):
        """Gera quiz"""
        if not self.last_transcription:
            self._update_status("❌ Processe um vídeo primeiro para gerar o quiz.", "error")
            return

        btn.disabled = True
        self._update_status("🎲 Gerando quiz personalizado...", "info")

        quiz = self.core.generate_quiz(self.last_transcription)

        with self.output:
            if quiz['success']:
                display(HTML(f"""
                <div class="section-box" style="background: #fef3c7; border-left-color: #f59e0b;">
                    <h3>❓ Quiz Gerado</h3>
                    <pre style="white-space: pre-wrap; line-height: 1.8;
                                font-family: inherit; background: white;
                                padding: 15px; border-radius: 8px;">{quiz['quiz']}</pre>
                </div>
                """))

        btn.disabled = False
        self._update_status("✅ Quiz gerado com sucesso!", "success")

    def _toggle_chat(self, btn):
        """Ativa/desativa modo chat"""
        if not self.last_transcription:
            self._update_status("❌ Processe um vídeo primeiro para conversar sobre ele.", "error")
            return

        self.chat_active = not self.chat_active

        if self.chat_active:
            # Ativar chat
            btn.description = '💬 Fechar Chat'
            btn.button_style = 'danger'
            self.chat_input.layout.display = 'flex'
            self.chat_send_btn.layout.display = 'flex'
            self.chat_output.layout.display = 'block'

            with self.chat_output:
                clear_output()
                display(HTML("""
                <div style="background: #f0f9ff; padding: 15px; border-radius: 8px;
                            border-left: 4px solid #3b82f6; margin: 10px 0;">
                    <strong>💬 Modo Conversação Ativado!</strong><br>
                    <p style="margin: 5px 0; color: #1e40af;">
                        Faça perguntas sobre o vídeo. Exemplos:<br>
                        • "Qual é o tema principal?"<br>
                        • "Resume os pontos mais importantes"<br>
                        • "O que aprendi com isso?"
                    </p>
                </div>
                """))
        else:
            # Desativar chat
            btn.description = '💬 Conversar sobre o Vídeo'
            btn.button_style = 'info'
            self.chat_input.layout.display = 'none'
            self.chat_send_btn.layout.display = 'none'
            self.chat_output.layout.display = 'none'

    def _send_chat(self, btn):
        """Envia mensagem no chat"""
        message = self.chat_input.value.strip()
        if not message:
            return

        btn.disabled = True
        self.chat_input.disabled = True

        with self.chat_output:
            display(HTML(f"""
            <div style="background: #e0f2fe; padding: 10px; border-radius: 8px;
                        margin: 5px 0; text-align: right;">
                <strong>Você:</strong> {message}
            </div>
            """))

            display(HTML("""
            <div style="background: #f1f5f9; padding: 10px; border-radius: 8px;
                        margin: 5px 0;">
                <strong>Tobias:</strong> <em>Pensando...</em>
            </div>
            """))

        # Gera resposta
        response = self.core.chat(message)

        with self.chat_output:
            # Remove "pensando..."
            clear_output(wait=True)

            # Mostra conversa completa
            for msg in self.core.conversation_history:
                display(HTML(f"""
                <div style="background: #e0f2fe; padding: 10px; border-radius: 8px;
                            margin: 5px 0; text-align: right;">
                    <strong>Você:</strong> {msg['user']}
                </div>
                <div style="background: #f1f5f9; padding: 10px; border-radius: 8px;
                            margin: 5px 0;">
                    <strong>Tobias:</strong> {msg['bot']}
                </div>
                """))

        self.chat_input.value = ''
        self.chat_input.disabled = False
        btn.disabled = False

    def _show_results(self, metadata: dict, transcription: str, analysis: dict):
        """Mostra resultados"""

        # Define contexto para conversação
        self.core.set_video_context(transcription, metadata)

        with self.output:
            clear_output()

            html = f"""
            <div class="section-box">
                <h2 style="color: #1e293b;">📹 {metadata['title']}</h2>
                <p style="color: #64748b; margin: 10px 0;">
                    <strong>Canal:</strong> {metadata['channel']} |
                    <strong>Views:</strong> {metadata['views']:,} |
                    <strong>Duração:</strong> {metadata['duration']//60}min {metadata['duration']%60}s
                </p>

                <hr style="border: none; border-top: 2px solid #e5e7eb; margin: 20px 0;">

                <h3 style="color: #475569;">📝 Transcrição ({len(transcription.split())} palavras)</h3>
                <details style="margin: 10px 0;">
                    <summary style="cursor: pointer; color: #667eea; font-weight: 600;
                                    padding: 10px; background: #f1f5f9; border-radius: 6px;">
                        📄 Clique para ver transcrição completa
                    </summary>
                    <div style="background: white; padding: 15px; border-radius: 8px;
                                margin-top: 10px; line-height: 1.8; max-height: 400px;
                                overflow-y: auto; border: 1px solid #e5e7eb;">
                        {transcription}
                    </div>
                </details>
            """

            if analysis['success']:
                html += f"""
                <hr style="border: none; border-top: 2px solid #e5e7eb; margin: 20px 0;">

                <div style="background: linear-gradient(135deg, #e0f2fe 0%, #bae6fd 100%);
                            padding: 20px; border-radius: 12px; margin: 15px 0;">
                    <div style="display: flex; justify-content: space-between; align-items: center; margin-bottom: 15px;">
                        <h3 style="color: #0369a1; margin: 0;">🤖 Análise Completa</h3>
                        <span style="background: #0369a1; color: white; padding: 6px 14px;
                                     border-radius: 15px; font-size: 12px; font-weight: 600;">
                            {analysis.get('provider', 'IA')}
                        </span>
                    </div>
                    <div style="background: white; padding: 20px; border-radius: 8px;
                                line-height: 1.8; white-space: pre-wrap; font-size: 15px;">
{analysis['analysis']}
                    </div>
                </div>
                """

            html += "</div>"
            display(HTML(html))

    def display(self):
        """Exibe interface"""
        layout = widgets.VBox([
            self.css,
            self.header,
            self.info,

            self.sep1,
            self.load_model_btn,

            self.sep2,
            self.mode_dropdown,

            self.sep3,
            widgets.HBox([self.url_input, self.process_btn]),

            self.sep4,
            widgets.HBox([self.quiz_btn, self.chat_btn]),

            # Área de chat
            self.space1,
            widgets.HBox([self.chat_input, self.chat_send_btn]),
            self.chat_output,

            self.space2,
            self.progress_label,
            self.progress,
            self.status,
            self.output
        ])

        display(layout)

display(HTML("""
<div style="background: #dbeafe; color: #1e40af; padding: 15px;
            border-radius: 8px; text-align: center;">
    ✅ TobiasInterface carregada | 👉 Execute a CÉLULA 6
</div>
"""))


In [ ]:
# =============================================================================
# 🚀 CÉLULA 6: INICIAR TOBIAS
# Execute por último para mostrar a interface
# =============================================================================

# Verificar se todas as classes foram carregadas
try:
    tobias = TobiasInterface()
    tobias.display()

    print("\n" + "="*70)
    print("🤖 TOBIAS AI ESTÁ PRONTO!")
    print("="*70)
    print("\n⚡ OTIMIZAÇÕES DE VELOCIDADE APLICADAS:")
    print("• 🚀 Download direto sem conversão (10-30 segundos)")
    print("• ⚡ Processamento simplificado e funcional")
    print("• 💨 Whisper otimizado")
    print("• 📊 Logs detalhados visíveis no console")
    print("\n💬 MODO CONVERSAÇÃO DISPONÍVEL!")
    print("• Converse com o Tobias sobre o vídeo processado")
    print("• Faça perguntas e receba respostas contextualizadas")
    print("\n📝 COMO USAR:")
    print("1. Clique em 'Carregar Modelo de IA' (só na primeira vez)")
    print("2. Escolha o modo de análise")
    print("3. Cole a URL do YouTube (vídeos até 20 minutos)")
    print("4. Clique em 'Processar Vídeo'")
    print("5. Acompanhe os logs no console abaixo")
    print("6. Após processar:")
    print("   • Gere quiz")
    print("   • Converse sobre o vídeo")
    print("\n💡 IMPORTANTE:")
    print("• Vídeos de 5-10 minutos processam em ~2-5 minutos")
    print("• Você verá TODOS os logs de progresso")
    print("• Se travar, verifique os logs de erro")
    print("="*70)

except NameError as e:
    print("="*70)
    print("❌ ERRO: Classes não carregadas!")
    print("="*70)
    print(f"\nErro: {e}\n")
    print("📝 SOLUÇÃO:")
    print("Execute as células NA ORDEM:")
    print("1. CÉLULA 1 - Instalação")
    print("2. CÉLULA 2 - Imports")
    print("3. CÉLULA 3 - TobiasCore")
    print("4. CÉLULA 4 - VideoProcessor")
    print("5. CÉLULA 5 - TobiasInterface")
    print("6. CÉLULA 6 - Iniciar (esta célula)")
    print("\n⚠️ Não pule nenhuma célula!")
    print("="*70) ("modo de análise desejado")
print("3. Cole a URL do YouTube")
print("4. Clique em 'Processar Vídeo'")
print("5. Após processar, você pode gerar um quiz!")
print("\n💡 DICA: Use vídeos de até 15 minutos para melhor performance")
print("="*70)

🚀 GPU detectada! Usando aceleração CUDA



🤖 TOBIAS AI ESTÁ PRONTO!

⚡ OTIMIZAÇÕES DE VELOCIDADE APLICADAS:
• 🚀 Download direto sem conversão (10-30 segundos)
• ⚡ Processamento simplificado e funcional
• 💨 Whisper otimizado
• 📊 Logs detalhados visíveis no console

💬 MODO CONVERSAÇÃO DISPONÍVEL!
• Converse com o Tobias sobre o vídeo processado
• Faça perguntas e receba respostas contextualizadas

📝 COMO USAR:
1. Clique em 'Carregar Modelo de IA' (só na primeira vez)
2. Escolha o modo de análise
3. Cole a URL do YouTube (vídeos até 20 minutos)
4. Clique em 'Processar Vídeo'
5. Acompanhe os logs no console abaixo
6. Após processar:
   • Gere quiz
   • Converse sobre o vídeo

💡 IMPORTANTE:
• Vídeos de 5-10 minutos processam em ~2-5 minutos
• Você verá TODOS os logs de progresso
• Se travar, verifique os logs de erro
3. Cole a URL do YouTube
4. Clique em 'Processar Vídeo'
5. Após processar, você pode gerar um quiz!

💡 DICA: Use vídeos de até 15 minutos para melhor performance
📥 Baixando modelo FLAN-T5... (primeira vez apenas)


Device set to use cuda:0


✅ Modelo carregado em GPU (CUDA)!

INICIANDO PROCESSAMENTO

[FASE 1] Download do YouTube
Limpando arquivos antigos...
Iniciando download de: https://www.youtube.com/watch?v=sfUM3gtuiUY
Extraindo informações...
[youtube] Extracting URL: https://www.youtube.com/watch?v=sfUM3gtuiUY
[youtube] sfUM3gtuiUY: Downloading webpage
[youtube] sfUM3gtuiUY: Downloading tv client config
[youtube] sfUM3gtuiUY: Downloading tv player API JSON
[youtube] sfUM3gtuiUY: Downloading web safari player API JSON
[youtube] sfUM3gtuiUY: Downloading m3u8 information
Baixando: Análise de circuitos com diodos (Exercício 2) (9min)
[youtube] Extracting URL: https://www.youtube.com/watch?v=sfUM3gtuiUY
[youtube] sfUM3gtuiUY: Downloading webpage
[youtube] sfUM3gtuiUY: Downloading tv client config
[youtube] sfUM3gtuiUY: Downloading tv player API JSON
[youtube] sfUM3gtuiUY: Downloading web safari player API JSON
[youtube] sfUM3gtuiUY: Downloading m3u8 information
[info] sfUM3gtuiUY: Downloading 1 format(s): 140
[download] S

100%|██████████| 56299/56299 [00:12<00:00, 4491.37frames/s]


...

100%|██████████| 56299/56299 [00:12<00:00, 4410.92frames/s]


✅ Transcrição: 849 palavras


Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



[FASE 3] Análise com IA

[FASE 4] Gerando resultados

[LIMPEZA] Removendo arquivos temporários
Removido: audio

PROCESSAMENTO CONCLUÍDO



The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Both `max_new_tokens` (=256) and `max_length`(=450) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
